In [1]:
%reset
import torch

import sys
sys.path.append('..')
from scripts.module import ply

import open3d as o3d
import numpy as np


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
# points_torch is in the sensor's local coordinate system, not yet transformed to the global system

# T0 = get_time()

dev = 0 # DEVICE 0，GPU的编号，不用管
surface_sample_range = 0.25 # noise level (actually as the std for a gaussian distribution) 
surface_sample_n = 3
freespace_behind_sample_n = 1
freespace_front_sample_n = 3
all_sample_n = surface_sample_n+freespace_behind_sample_n+freespace_front_sample_n+1 # 1 as the exact measurement
# all_sample_n = surface_sample_n + freespace_front_sample_n + 1 # 法向量-guided的采样 + SDF场之前的点 + 1(测量点云)
free_front_min_ratio = 0.7
free_sample_end_dist = 0.6
sigma_base = 0.08

aa = ply.read_ply('/home/server/PIN_SLAM/data/wudasuidao/normal/40frame/ply_normalcorrect/1724123355371955884.ply')

points_torch = torch.tensor(np.vstack((aa['x'] , aa['y'], aa['z'])).T , device=dev)
normal_torch = torch.tensor(np.vstack((aa['normal_x'] , aa['normal_y'], aa['normal_z'])).T , device=dev)

In [5]:
# get sample points
point_num = points_torch.shape[0] # 82976
distances = torch.linalg.norm(points_torch, dim=1, keepdim=True) # ray distances (scaled) # [82976,1] 比较大的一个数[[39.5087, 39.6541, ..., 57.9294, 58.5979, 57.5824]]

# Part 0. the exact measured point
measured_sample_displacement = torch.zeros_like(distances) # [82976,1]
measured_sample_dist_ratio = torch.ones_like(distances) # [82976,1]

# Part 1. close-to-surface uniform sampling 
# uniform sample in the close-to-surface range (+- range)
surface_sample_displacement = torch.randn(point_num*surface_sample_n, 1, device=dev)*surface_sample_range # [248928,1] = [82976*3,1]

repeated_dist = distances.repeat(surface_sample_n,1) # [248928,1] = [82976*3,1]
surface_sample_dist_ratio = surface_sample_displacement/repeated_dist + 1.0 # 1.0 means on the surface [248928,1]


# Part 2. free space (in front of surface) uniform sampling
# if you want to reconstruct the thin objects (like poles, tree branches) well, you need more freespace samples to have 
# a space carving effect

sigma_ratio = 2.0
repeated_dist = distances.repeat(freespace_front_sample_n,1) # [165952,1] = [82976*2,1]
free_max_ratio = 1.0 - sigma_ratio * surface_sample_range / repeated_dist
free_diff_ratio = free_max_ratio - free_front_min_ratio
free_sample_front_dist_ratio = torch.rand(point_num*freespace_front_sample_n, 1, device=dev)*free_diff_ratio + free_front_min_ratio # [165952,1]
free_sample_front_displacement = (free_sample_front_dist_ratio - 1.0) * repeated_dist # [165952,1]


# Part 3. free space (behind surface) uniform sampling
repeated_dist = distances.repeat(freespace_behind_sample_n,1) # [82976,1]
free_max_ratio = free_sample_end_dist / repeated_dist + 1.0
free_behind_min_ratio = 1.0 + sigma_ratio * surface_sample_range / repeated_dist
free_diff_ratio = free_max_ratio - free_behind_min_ratio

free_sample_behind_dist_ratio = torch.rand(point_num*freespace_behind_sample_n, 1, device=dev)*free_diff_ratio + free_behind_min_ratio # [82976,1]

free_sample_behind_displacement = (free_sample_behind_dist_ratio - 1.0) * repeated_dist # [82976,1]


# T1 = get_time()

# all together
all_sample_displacement = torch.cat((measured_sample_displacement, surface_sample_displacement, free_sample_front_displacement, free_sample_behind_displacement),0) # [580832,1]=[82976,1]+[248928,1]+[165952,1]+[82976,1]
all_sample_dist_ratio = torch.cat((measured_sample_dist_ratio, surface_sample_dist_ratio, free_sample_front_dist_ratio, free_sample_behind_dist_ratio),0) # [580832,1]=[82976,1]+[248928,1]+[165952,1]+[82976,1]

repeated_points = points_torch.repeat(all_sample_n,1) # [580832,3]
repeated_dist = distances.repeat(all_sample_n,1) # [580832,1]
all_sample_points = repeated_points*all_sample_dist_ratio # [580832,3]

# depth tensor of all the samples
depths_tensor = repeated_dist * all_sample_dist_ratio # [580832,1]

# get the weight vector as the inverse of sigma
weight_tensor = torch.ones_like(depths_tensor) # [580832,1]

surface_sample_count = point_num*(surface_sample_n+1) # 331904=82976*(3+1)
if True: # far away surface samples would have lower weight
    weight_tensor[:surface_sample_count] = 1 + 0.5*0.5 - (repeated_dist[:surface_sample_count] / 40) * 0.5 # [0.6, 1.4]
# TODO: also add lower weight for surface samples with large incidence angle

# behind surface weight drop-off because we have less uncertainty behind the surface
if False:
    dropoff_min = 0.2 * free_sample_end_dist
    dropoff_max = free_sample_end_dist
    dropoff_diff = dropoff_max - dropoff_min
    # behind_displacement = (repeated_dist*(all_sample_dist_ratio-1.0)/sigma_base)
    behind_displacement = all_sample_displacement
    dropoff_weight = (dropoff_max - behind_displacement) / dropoff_diff
    dropoff_weight = torch.clamp(dropoff_weight, min = 0.0, max = 1.0)
    dropoff_weight = dropoff_weight * 0.8 + 0.2
    weight_tensor = weight_tensor * dropoff_weight

# give a flag indicating the type of the sample [negative: freespace, positive: surface]
weight_tensor[surface_sample_count:] *= -1.0 

# ray-wise depth
distances = distances.squeeze(1)

# assign sdf labels to the samples
# projective distance as the label: behind +, in-front - 
sdf_label_tensor = all_sample_displacement.squeeze(1)  # scaled [-1, 1] # as distance (before sigmoid)# [580832]

# assign the normal label to the samples
normal_label_tensor = None
if normal_torch is not None:
    normal_label_tensor = normal_torch.repeat(all_sample_n,1)

# assign the semantic label to the samples (including free space as the 0 label)
sem_label_tensor = None

# assign the color label to the close-to-surface samples
color_tensor = None


# T2 = get_time()
# Convert from the all ray surface + all ray free order to the ray-wise (surface + free) order
all_sample_points = all_sample_points.reshape(all_sample_n, -1, 3).transpose(0, 1).reshape(-1, 3) # [580832,3] -> [580832,3]
sdf_label_tensor = sdf_label_tensor.reshape(all_sample_n, -1).transpose(0, 1).reshape(-1) # [580832]
sdf_label_tensor *= (-1) # convert to the same sign as 

weight_tensor = weight_tensor.reshape(all_sample_n, -1).transpose(0, 1).reshape(-1) # [580832,1] -> [580832]
# depths_tensor = depths_tensor.reshape(all_sample_n, -1).transpose(0, 1).reshape(-1)

if normal_torch is not None:
    normal_label_tensor = normal_label_tensor.reshape(all_sample_n, -1, 3).transpose(0, 1).reshape(-1, 3)

# ray distance (distances) is not repeated

# T3 = get_time()

# print("time for sampling I:", T1-T0)
# print("time for sampling II:", T2-T1)
# print("time for sampling III:", T3-T2)
# all super fast, all together in 0.5 ms

In [9]:


pc_load=o3d.geometry.PointCloud()
pc_load.points= o3d.utility.Vector3dVector(all_sample_points.cpu().numpy())
# pc_load.normals = o3d.utility.Vector3dVector(normals)


o3d.visualization.draw_geometries([pc_load],
                                # zoom=0.3412,
                                # front=[0.4257, -0.2125, -0.8795],
                                # lookat=[2.6172, 2.0475, 1.532],
                                # up=[-0.0694, -0.9768, 0.2024],
                                point_show_normal=True
                                )